In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/airbnb_listings.csv')    # read the csv data file to a Dataframe

In [3]:
df.shape    # data has 14861 rows and 18 columns

(14861, 18)

In [4]:
df.duplicated().sum()   # there are no duplicate rows in the dataset

0

In [5]:
df.head()   # quick look at the dataset

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,5456,Guesthouse in Austin · ★4.84 · 1 bedroom · 2 b...,8028,Sylvia,NaN,78702,30.26057,-97.73441,Entire home/apt,126,2,657,2023-08-27,3.72,1,306,42,NaN
1,5769,Home in Austin · ★4.90 · 1 bedroom · 1 bed · 1...,8186,Elizabeth,NaN,78729,30.45697,-97.78422,Private room,45,1,290,2023-08-04,1.77,1,0,21,NaN
2,6413,Guesthouse in Austin · ★4.97 · Studio · 1 bed ...,13879,Todd,NaN,78704,30.24885,-97.73587,Entire home/apt,57,30,122,2022-10-17,0.73,1,0,3,NaN
3,6448,Guesthouse in Austin · ★4.97 · 1 bedroom · 2 b...,14156,Amy,NaN,78704,30.26034,-97.76487,Entire home/apt,159,3,305,2023-09-04,2.09,1,156,17,NaN
4,8502,Guest suite in Austin · ★4.56 · 1 bedroom · 1 ...,25298,Karen,NaN,78741,30.23466,-97.73682,Entire home/apt,48,4,51,2023-05-16,0.31,1,98,3,NaN


### Data Cleaning and Transformation

In [6]:
# check the data types and null values count in every column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14861 entries, 0 to 14860
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              14861 non-null  int64  
 1   name                            14861 non-null  object 
 2   host_id                         14861 non-null  int64  
 3   host_name                       14859 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   14861 non-null  int64  
 6   latitude                        14861 non-null  float64
 7   longitude                       14861 non-null  float64
 8   room_type                       14861 non-null  object 
 9   price                           14861 non-null  int64  
 10  minimum_nights                  14861 non-null  int64  
 11  number_of_reviews               14861 non-null  int64  
 12  last_review                     

In [7]:
# All rows have null value in the neighbourhood group and license field so remove these fields
df = df.drop(['neighbourhood_group', 'license'], axis=1)
df.head(2)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,5456,Guesthouse in Austin · ★4.84 · 1 bedroom · 2 b...,8028,Sylvia,78702,30.26057,-97.73441,Entire home/apt,126,2,657,2023-08-27,3.72,1,306,42
1,5769,Home in Austin · ★4.90 · 1 bedroom · 1 bed · 1...,8186,Elizabeth,78729,30.45697,-97.78422,Private room,45,1,290,2023-08-04,1.77,1,0,21


In [8]:
# Remove the two rows without a host name

df = df[~df['host_name'].isna()]

In [9]:
# drop the rows which do not have last review date or reviews per month

df = df.dropna(subset=['number_of_reviews', 'last_review'], axis=0)

In [10]:
df.columns[df.isna().sum() != 0] # no null values in any field

Index([], dtype='object')

In [11]:
df.drop('id', axis=1, inplace=True)   # remove the id (airbnb id) field. It is of no use

In [12]:
# change the name of the price field to daily_price (to avoid confusion)

df.rename(columns={'price': 'daily_price'}, inplace=True)

In [13]:
# Split the name field into 5 different fields:

# property_type (object)
# bedrooms (int64)
# beds (int64)
# bathrooms (int64)
# ratings (float64)

# Check rows which do not contain sufficient info to split the name into 5 fields
(df['name'].str.split(' · ').apply(len) < 5).sum()

1886

In [14]:
# remove those rows
df = df[df['name'].str.split(' · ').apply(len) == 5]

In [15]:
# a sample view of the list after splitting
names = df['name'].str.split(' · ')
for i in range(5):
    print(names[i])


['Guesthouse in Austin', '★4.84', '1 bedroom', '2 beds', '1 bath']
['Home in Austin', '★4.90', '1 bedroom', '1 bed', '1 shared bath']
['Guesthouse in Austin', '★4.97', 'Studio', '1 bed', '1 bath']
['Guesthouse in Austin', '★4.97', '1 bedroom', '2 beds', '1 bath']
['Guest suite in Austin', '★4.56', '1 bedroom', '1 bed', '1 bath']


In [16]:
# create and assign the values to those fields
property_type, ratings, bedrooms, beds, bathrooms = [], [], [], [], []

for name in names:
    property_type.append(name[0].split(' in ')[0])
    ratings.append(name[1][1:])
    bedrooms.append(name[2].split(' ')[0])
    beds.append(name[3].split(' ')[0])
    bathrooms.append(name[4].split(' ')[0])

df['property_type'] = property_type
df['bedrooms'] = bedrooms
df['beds'] = beds
df['bathrooms'] = bathrooms
df['ratings'] = ratings

In [17]:
# drop the original name field
df.drop('name', axis=1, inplace=True)

In [18]:
df.head(2)

,host_id,host_name,neighbourhood,latitude,longitude,room_type,daily_price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,property_type,bedrooms,beds,bathrooms,ratings
0,8028,Sylvia,78702,30.26057,-97.73441,Entire home/apt,126,2,657,2023-08-27,3.72,1,306,42,Guesthouse,1,2,1,4.84
1,8186,Elizabeth,78729,30.45697,-97.78422,Private room,45,1,290,2023-08-04,1.77,1,0,21,Home,1,1,1,4.90


In [19]:
# reposition the columns and bring them to the front
column_order = list(df.columns[:2]) + list(df.columns[-5:]) + list(df.columns[2:-5])
df = df[column_order]

In [20]:
# repostitioned columns dataframe
df.head(2)

,host_id,host_name,property_type,bedrooms,beds,bathrooms,ratings,neighbourhood,latitude,longitude,room_type,daily_price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,8028,Sylvia,Guesthouse,1,2,1,4.84,78702,30.26057,-97.73441,Entire home/apt,126,2,657,2023-08-27,3.72,1,306,42
1,8186,Elizabeth,Home,1,1,1,4.90,78729,30.45697,-97.78422,Private room,45,1,290,2023-08-04,1.77,1,0,21


In [21]:
# check to see if the data is in the wrong format in any fields
def is_numeric_string(value):
    decimal_count = 0

    for char in value:
        if char.isdigit():
            continue
        elif char == '.' and decimal_count == 0:
            decimal_count += 1
        else:
            return False

    return True

df[~df['bedrooms'].apply(is_numeric_string)]['bedrooms'].value_counts()

bedrooms
Studio    485
Name: count, dtype: int64

In [22]:
# Change 'Studio' to 0.5 bedroom for numeric coversion later on

df.loc[df.bedrooms == 'Studio', 'bedrooms'] = '0.5'

In [23]:
# check for beds field

df[~df['beds'].apply(is_numeric_string)]['beds'].value_counts()  # all values are in proper format

Series([], Name: count, dtype: int64)

In [24]:
# check for bathrooms

df[~df['bathrooms'].apply(is_numeric_string)]['bathrooms'].value_counts()

bathrooms
Half-bath    3
Private      1
Shared       1
Name: count, dtype: int64

In [25]:
# replace half-bath with 0.5 and remove the others

df.loc[df['bathrooms'] == 'Half-bath', 'bathrooms'] = '0.5'
df = df[df['bathrooms'].apply(is_numeric_string)]
df[~df['bathrooms'].apply(is_numeric_string)]['bathrooms'].value_counts()

Series([], Name: count, dtype: int64)

In [26]:
# check for the ratings field

df[~df['ratings'].apply(is_numeric_string)]['ratings'].value_counts()

ratings
New    257
Name: count, dtype: int64

In [27]:
# remove the rows with 'New' value in the ratings field

df = df[df['ratings'].apply(is_numeric_string)]
df[~df['ratings'].apply(is_numeric_string)]['ratings'].value_counts()

Series([], Name: count, dtype: int64)

In [28]:
# removing the rows with value 'Bed and breakfast' in property_type field. It is not a specific type
df = df[df['property_type'] != 'Bed and breakfast']

In [29]:
# Custom function to apply different data type conversions
def custom_conversion(value):
    try:
        # Try converting to integer
        return int(value)
    except ValueError:
        try:
            # Try converting to float
            return float(value)
        except ValueError:
            # Return NaN for inconvertibles
            return float('nan')


# finally change the data types of the new fields
df[['bedrooms', 'beds', 'bathrooms', 'ratings']] = df[['bedrooms', 'beds', 'bathrooms', 'ratings']].map(custom_conversion)

In [30]:
# all the data is in proper format now

df.shape    # It has 9600 rows and 19 columns

(9600, 19)

In [31]:
df.head()

,host_id,host_name,property_type,bedrooms,beds,bathrooms,ratings,neighbourhood,latitude,longitude,room_type,daily_price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,8028,Sylvia,Guesthouse,1.0,2,1.0,4.84,78702,30.26057,-97.73441,Entire home/apt,126,2,657,2023-08-27,3.72,1,306,42
1,8186,Elizabeth,Home,1.0,1,1.0,4.90,78729,30.45697,-97.78422,Private room,45,1,290,2023-08-04,1.77,1,0,21
2,13879,Todd,Guesthouse,0.5,1,1.0,4.97,78704,30.24885,-97.73587,Entire home/apt,57,30,122,2022-10-17,0.73,1,0,3
3,14156,Amy,Guesthouse,1.0,2,1.0,4.97,78704,30.26034,-97.76487,Entire home/apt,159,3,305,2023-09-04,2.09,1,156,17
4,25298,Karen,Guest suite,1.0,1,1.0,4.56,78741,30.23466,-97.73682,Entire home/apt,48,4,51,2023-05-16,0.31,1,98,3


In [32]:
# a final info of the cleaned and transformed data
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9600 entries, 0 to 14597
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   host_id                         9600 non-null   int64  
 1   host_name                       9600 non-null   object 
 2   property_type                   9600 non-null   object 
 3   bedrooms                        9600 non-null   float64
 4   beds                            9600 non-null   int64  
 5   bathrooms                       9600 non-null   float64
 6   ratings                         9600 non-null   float64
 7   neighbourhood                   9600 non-null   int64  
 8   latitude                        9600 non-null   float64
 9   longitude                       9600 non-null   float64
 10  room_type                       9600 non-null   object 
 11  daily_price                     9600 non-null   int64  
 12  minimum_nights                  9600 n

In [33]:
# finally extract the cleaned data to a csv file

df.to_csv('airbnb_listings_clean_data.csv', index=False)